In [70]:
import pandas as pd
import numpy as np
import math
import datetime
import tqdm as tqdm

In [71]:
delete_month = True
del_date = pd.to_datetime('2021-11-30')

In [72]:
cases = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_cases.csv')

if delete_month:
    cases = cases[pd.to_datetime(cases['date_time']) < del_date ]
    cases.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3/short/customer_cases.csv', index=False, encoding='utf-8', sep=',')
else: 
    cases.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3/full/customer_cases.csv', index=False, encoding='utf-8', sep=',')


print(len(cases))
print(cases['reason'].unique())
print(cases['channel'].unique())
cases.sort_values(by='date_time')

1297621
['signup' 'support']
['phone' 'email']


,case_id,date_time,customer_id,channel,reason
451489,CC451590,2017-01-01 00:04:18,C632357,email,signup
731434,CC731535,2017-01-01 00:59:52,C912302,email,support
788639,CC788740,2017-01-01 01:01:17,C969507,phone,signup
827922,CC828023,2017-01-01 01:06:56,C1008790,phone,signup
622777,CC622878,2017-01-01 01:19:05,C803645,phone,support
...,...,...,...,...,...
422787,CC422888,2021-11-29 23:36:49.432662,C603655,phone,support
1153460,CC1153561,2021-11-29 23:39:40.556314,C1334328,phone,support
671810,CC671911,2021-11-29 23:52:11.310676,C852678,phone,support
515028,CC515129,2021-11-29 23:53:47.836258,C695896,phone,signup


In [73]:
dim_geo = pd.read_excel(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\dim_geo.xlsx')
dim_geo.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\dim_geo.csv', sep=',', encoding='utf-8', index=False)
dim_geo.head()

,geo_id,Kommune,Region
0,1,Albertslund,Region Hovedstaden
1,2,Allerød,Region Hovedstaden
2,3,Ballerup,Region Hovedstaden
3,4,Bornholm,Region Hovedstaden
4,5,Brøndby,Region Hovedstaden


In [74]:
dim_customer = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_info.csv')

# Add age group
groups = [10,20,30,40,50,60,70,80,90]
dim_customer['age_group'] = pd.cut(dim_customer['age'], bins=groups, right=True).astype(str).str.replace(', ','-', regex=True)

# Add geography
dim_customer['geo_id'] = pd.Series(np.random.randint(1, 98, size=len(dim_customer))).astype('Int64')

# delete month
if delete_month:
    dim_customer = dim_customer[dim_customer['customer_id'].isin(cases['customer_id'].unique())]
    dim_customer.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\short/dim_customer.csv', sep=',', index=False, encoding='utf-8', )
else:
    dim_customer.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\full/dim_customer.csv', sep=',', index=False, encoding='utf-8', )

print(len(dim_customer))
dim_customer.head()

1229122


,customer_id,age,gender,age_group,geo_id
0,C2448,76,female,(70-80],14
1,C2449,61,male,(60-70],88
2,C2450,58,female,(50-60],24
3,C2451,62,female,(60-70],29
4,C2452,71,male,(70-80],37


In [75]:
male = dim_customer.iloc[np.where(dim_customer['gender']== 'male')].customer_id.copy()
male

1             C2449
4             C2452
5             C2453
6             C2454
9             C2457
             ...   
1508925    C1511373
1508927    C1511375
1508928    C1511376
1508929    C1511377
1508931    C1511379
Name: customer_id, Length: 561895, dtype: object

In [76]:
product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\customer_product.csv')
product['cancel_date_time'] = product['cancel_date_time'].replace(np.nan, '1/1/2100').astype(np.datetime64).copy()
product['signup_date_time'] = product['signup_date_time'].astype(np.datetime64).copy()
product['signup_month'] = product['signup_date_time'].dt.to_period('M').copy()
product['cancel_month'] = product['cancel_date_time'].dt.to_period('M').copy()

#print('1:', product.columns)
# Overwrite to make old guys use product 1
#old_lst = dim_customer.iloc[np.where(dim_customer['age']>60)].customer_id.sample(300000, random_state=741).index
#product.iloc[old_lst, 1] = 'prd_1'


#print('2:', product.columns)
# Overwrite to make young guys use product 2
#young = dim_customer.iloc[np.where(dim_customer['age']<45)].customer_id.copy()
#young_lst = young.sample(22000, random_state=741).to_numpy()
#product['product'][product['customer_id'].isin(young_lst)] = 'prd_2'


#print('3:', product.columns)
# Overwrite to make Male use product 1
#male = dim_customer.iloc[np.where(dim_customer['gender']== 'male')].customer_id.copy()
#male_lst = male.sample(300000, random_state=741).to_numpy()
#product['product'][product['customer_id'].isin(male_lst)] = 'prd_1'

# delete month
if delete_month:
    product = product[product['customer_id'].isin(cases['customer_id'].unique())]
    product[['customer_id', 'product', 'signup_date_time', 'cancel_date_time']].to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\short/fact_product.csv', sep=',', index=False, encoding='utf-8')
else:
    product[['customer_id', 'product', 'signup_date_time', 'cancel_date_time']].to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\full/fact_product.csv', sep=',', index=False, encoding='utf-8')


print(len(product))
product.head()

1229122


,customer_id,product,signup_date_time,cancel_date_time,signup_month,cancel_month
0,C2448,prd_1,2017-01-01 10:35:09,2100-01-01 00:00:00,2017-01,2100-01
1,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,2017-01,2021-09
2,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,2017-01,2019-01
3,C2451,prd_2,2017-01-01 13:32:08,2100-01-01 00:00:00,2017-01,2100-01
4,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,2017-01,2021-06


In [77]:
graf = product.set_index('customer_id').join(dim_customer.set_index('customer_id'))
graf.groupby(['age_group', 'product', 'gender'])['age'].count()

age_group  product  gender
(20-30]    prd_1    female       959
                    male        1309
           prd_2    female       607
                    male         833
(30-40]    prd_1    female      7886
                    male       11054
           prd_2    female      4828
                    male        6774
(40-50]    prd_1    female     43787
                    male       64946
           prd_2    female     26807
                    male       40062
(50-60]    prd_1    female    132987
                    male      174267
           prd_2    female     80153
                    male      106952
(60-70]    prd_1    female    199677
                    male       79832
           prd_2    female    118359
                    male       53223
(70-80]    prd_1    female     32124
                    male       13642
           prd_2    female     19053
                    male        9001
Name: age, dtype: int64

In [78]:
def num_active_user_in_a_period(start_date):
    '''count active users'''
    number_of_active =  sum((product['signup_month'] <= start_date) & (product['cancel_month'] >= start_date))
    return number_of_active


def who_is_active_in_period(start_date):
    '''count active users'''
    active_users =  (product['signup_month'] <= start_date) & (product['cancel_month'] >= start_date)
    return product[active_users]['customer_id'].to_numpy()


def who_exists(start_date):
    '''count active users'''
    active_users =  (product['signup_month'] <= start_date) 
    return product[active_users]['customer_id'].reset_index(drop=True).to_numpy()


In [79]:
fact_active = pd.DataFrame()
end_date =  cases.date_time.max()
date_list = pd.date_range(start='1/1/2017', end=end_date).to_period('M').unique().to_numpy()

# List of customers who signed up and caceled by month
signup = product.groupby(['signup_month'])['customer_id'].apply(list)
cancel = product.groupby(['cancel_month'])['customer_id'].apply(list)   


# Generate fact table
for d in tqdm.tqdm(date_list[:-1]):
    
    # Get users who have been created at time d
    _users =  who_exists(d)

    # store date d, as np.array
    dates = np.repeat(d, len(_users))

    # Zip/Stack dates and users
    _df = pd.DataFrame(np.dstack((dates,_users))[0], columns = ['date','customer_id']).set_index('date')

    # is_active column
    _df['active'] = _df['customer_id'].isin(who_is_active_in_period(d))

    # new customer this month
    _df['new_customer'] = _df['customer_id'].isin(signup[d])

    # custumers who canceled
    _df['cancel'] = _df['customer_id'].isin(cancel[d])

    # Append these users 
    fact_active = pd.concat([fact_active,_df], ignore_index=False)

fact_active.index = fact_active.index.strftime('%Y-%m-%d')


if delete_month:
    fact_active = fact_active[fact_active['customer_id'].isin(cases['customer_id'].unique())]
    fact_active.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\short/fact_active.csv', encoding='utf-8', sep=',', index=True)
else:
    fact_active.to_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\to_s3\full/fact_active.csv', encoding='utf-8', sep=',', index=True)



print(fact_active)

100%|██████████| 58/58 [00:37<00:00,  1.56it/s]


           customer_id  active  new_customer  cancel
date                                                
2017-01-31       C2448    True          True   False
2017-01-31       C2449    True          True   False
2017-01-31       C2450    True          True   False
2017-01-31       C2451    True          True   False
2017-01-31       C2452    True          True   False
...                ...     ...           ...     ...
2021-10-31    C1511373   False         False   False
2021-10-31    C1511375    True         False    True
2021-10-31    C1511376    True         False   False
2021-10-31    C1511377    True          True   False
2021-10-31    C1511378   False         False   False

[24656862 rows x 4 columns]


In [80]:
dim_product = pd.read_csv(r'C:\Users\MartinBirkAndreasen\Desktop\Jazz\QlikWorld\subscription\raw\product_info.csv')
dim_product.head()

,product_id,name,price,billing_cycle
0,prd_1,annual_subscription,1200,12
1,prd_2,monthly_subscription,125,1
